In [1]:
# Principales
import pandas as pd
import numpy as np

# Modificaciones 
import re
from unidecode import unidecode
import unicodedata
import datetime
import os


# visuzalizaciones
import missingno as msno
import matplotlib as plt 
import plotly.express as px 

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows',80)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_colwidth', None)
# Ajustar la configuración para mostrar más filas 
#pd.set_option('display.max_rows', 500)

#inicializo la lista to replace
to_replace=['Null','Nan','nan','NULL',None]

# Funciones

# 1. Carga de datos

In [5]:
dfEDA= pd.read_csv('./data/pisosEDA.csv',index_col=False)

C:\Users\extas\AppData\Local\Temp\ipykernel_16580\2743511520.py:1: DtypeWarning: Columns (0,1,15,52,62,76) have mixed types. Specify dtype option on import or set low_memory=False.
  dfEDA= pd.read_csv('./data/pisosEDA.csv',index_col=False)


In [8]:
dfEDA.shape

(10427, 77)

In [6]:
dfEDA.columns


Index(['precio', 'descuento', 'm2_constr', 'cod_distrito', 'cod_barrio',
       'letrace', 'tipologia', 'estado', 'amueblado', 'planta', 'dormitorios',
       'banos', 'terraza', 'garaje', 'trastero', 'ascensor', 'porteria',
       'calefaccion', 'ano_construccion', 'plantas_edificio',
       'aire_acondicionado', 'jardin', 'inmueble_ingresos',
       'alquiler_opcion_a_compra', 'disponibilidad', 'nuda_propiedad',
       'tipo_inmueble', 'tiene_armario', 'cancha_tenis',
       'carpinteria_exterior_doble_vidrio/pvc',
       'carpinteria_exterior_doble_vidrio/madera',
       'carpinteria_exterior_doble_vidrio/metal',
       'carpinteria_exterior_triple_vidrio/madera',
       'carpinteria_exterior_triple_vidrio/metal',
       'carpinteria_exterior_vidrio/pvc', 'carpinteria_exterior_vidrio/madera',
       'carpinteria_exterior_vidrio/metal', 'chimenea', 'cocina', 'exterior',
       'interior', 'interior_y_exterior', 'tiene_piscina', 'porton_electrico',
       'puerta_blindada', 'alarma', 

In [7]:
dfEDA.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
precio,10427.0,2541.0,650000.0,56.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
descuento,54,35,"-5,0%",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN
m2_constr,10400.0,NaN,NaN,NaN,137.21093,148.638886,0.0,68.0,98.0,149.0,4574.0
cod_distrito,10425.0,NaN,NaN,NaN,7.90753,5.757064,1.0,4.0,6.0,12.0,21.0
cod_barrio,10189.0,NaN,NaN,NaN,82.151536,57.63177,1.0,41.0,66.0,125.0,215.0
...,...,...,...,...,...,...,...,...,...,...,...
orientacion_sur,10427.0,NaN,NaN,NaN,0.046418,0.210399,0.0,0.0,0.0,0.0,1.0
sistema_alarma,281.0,NaN,NaN,NaN,0.017794,0.132436,0.0,0.0,0.0,0.0,1.0
grupo_cocina,10427,23,sin_informacion,8835,NaN,NaN,NaN,NaN,NaN,NaN,NaN
clasificacion_aire,10427,4,Desconocido,5943,NaN,NaN,NaN,NaN,NaN,NaN,NaN
